# Example with ARGO data

We first need to download some ARGO data for example.

In [1]:
import requests
url = 'https://www.ncei.noaa.gov/data/oceans/argo/gadr/data/coriolis/69022/nodc_69022_prof.nc'
r = requests.get(url, allow_redirects=True)
with open('ARGO_example.nc', 'wb') as f:
    f.write(r.content)

In [2]:
import xarray as xr
import cf_xarray
import gsw_xarray as gsw

In [3]:
ds = xr.open_dataset('ARGO_example.nc')

In [4]:
ds

<xarray.Dataset>
Dimensions:                       (n_prof: 130, n_param: 3, n_levels: 56,
                                   n_calib: 1, n_history: 0)
Dimensions without coordinates: n_prof, n_param, n_levels, n_calib, n_history
Data variables: (12/65)
    data_type                     object ...
    format_version                object ...
    handbook_version              object ...
    reference_date_time           object ...
    date_creation                 object ...
    date_update                   object ...
    ...                            ...
    history_parameter             (n_history, n_prof) object ...
    history_start_pres            (n_history, n_prof) float32 ...
    history_stop_pres             (n_history, n_prof) float32 ...
    history_previous_value        (n_history, n_prof) float32 ...
    history_qctest                (n_history, n_prof) object ...
    crs                           int32 ...
Attributes: (12/49)
    title:                           Argo float vertical profile
    institution:                     FR GDAC
    source:                          Argo float
    history:                         2019-10-20T09:53:12Z boyer convAGDAC.f90...
    references:                      https://www.nodc.noaa.gov/argo/
    user_manual_version:             3.1
    ...                              ...
    time_coverage_end:               2005-09-19T02:13:46Z
    time_coverage_duration:          point
    time_coverage_resolution:        point
    gadr_ConventionVersion:          GADR-3.0
    gadr_program:                    convAGDAC.f90
    gadr_programVersion:             1.2

We can rely on cf-xarray to see what variables have standard names in our dataset:

In [5]:
ds.cf

Coordinates:
- CF Axes:   X, Y, Z, T: n/a

- CF Coordinates:   longitude, latitude, vertical, time: n/a

- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

- Bounds:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   latitude: ['latitude']
                    longitude: ['longitude']
                    sea_water_pressure: ['pres', 'pres_adjusted']
                    sea_water_salinity: ['psal', 'psal_adjusted']
                    sea_water_temperature: ['temp', 'temp_adjusted']
                    time: ['juld']

- Bounds:   n/a

The dataset contains multiple time the same variable (e.g. 'pres_adjusted' and 'pres' both have the standard name 'sea_water_pressure'). For the accessor to work, only 1 variable or each standard name must be present, explicitely stated when calling the function, or the gsw option `set_cf_name_preference` must be set.
For this example we will only retain the adjusted variables. We set the global option, but we could also use context, i.e.

In [6]:
with gsw.set_cf_name_preference(
    sea_water_pressure="pres_adjusted",
    sea_water_practical_salinity="psal_adjusted",
    sea_water_temperature="temp_adjusted"
):
    # do the computation
    pass

gsw.set_cf_name_preference(
    sea_water_pressure="pres_adjusted",
    sea_water_practical_salinity="psal_adjusted",
    sea_water_temperature="temp_adjusted"
)

# We can check the options we have set:
gsw.get_options()

{'non_cf_name': {},
 'cf_name_preference': {'sea_water_pressure': 'pres_adjusted',
  'sea_water_practical_salinity': 'psal_adjusted',
  'sea_water_temperature': 'temp_adjusted'}}

In the following sections we will demonstrate each features of gsw-xarray. We will focus on computing the potential density anomaly.

## Basic usage as drop in replacement of gsw

In [7]:
gsw.sigma0?

Signature: gsw.sigma0(SA, CT)
Docstring:
Calculates potential density anomaly with reference pressure of 0 dbar,
this being this particular potential density minus 1000 kg/m^3.  This
function has inputs of Absolute Salinity and Conservative Temperature.
This function uses the computationally-efficient expression for
specific volume in terms of SA, CT and p (Roquet et al., 2015).

Parameters
----------
SA : array-like
    Absolute Salinity, g/kg
CT : array-like
    Conservative Temperature (ITS-90), degrees C

Returns
-------
sigma0 : array-like, kg/m^3
    potential density anomaly with
    respect to a reference pressure of 0 dbar,
    that is, this potential density - 1000 kg/m^3.
File:      ~/.cache/pypoetry/virtualenvs/gsw-xarray-NsrEXKiZ-py3.10/lib/python3.10/site-packages/gsw/_wrapped_ufuncs.py
Type:      function

We need Absolute Salinity and Conservative Temperature, so 1st we need to do some conversions:

In [8]:
SA = gsw.SA_from_SP(SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude)
CT = gsw.CT_from_t(SA=SA, t=ds.temp_adjusted, p=ds.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)
sigma0

<xarray.DataArray 'sigma0' (n_prof: 130, n_levels: 56)>
array([[26.64758465, 26.64656536, 26.64667857, ..., 27.77078682,
        27.77467073, 27.77756609],
       [26.48498586, 26.48409899, 26.48432005, ..., 27.76600562,
        27.76941237, 27.76895082],
       [26.25518807, 26.25449663, 26.25488776, ..., 27.7603578 ,
        27.76478937, 27.76924889],
       ...,
       [25.31136543, 25.31674362, 25.42998838, ..., 27.77414777,
        27.76859076,         nan],
       [25.30657435, 25.30409728, 25.29751108, ..., 27.78551733,
        27.77778253,         nan],
       [25.43166004, 25.43155161, 25.43251587, ..., 27.7946225 ,
        27.79664872,         nan]])
Dimensions without coordinates: n_prof, n_levels
Attributes:
    standard_name:  sea_water_sigma_t
    units:          kg/m^3

## Using Pint and pint-xarray to handle units

In [9]:
import pint_xarray
import cf_xarray.units

/home/romain/.cache/pypoetry/virtualenvs/gsw-xarray-NsrEXKiZ-py3.10/lib/python3.10/site-packages/cf_xarray/units.py:112: UserWarning: Import(s) unavailable to set up matplotlib support...skipping this portion of the setup.
  warnings.warn(


In [10]:
ds_pint = ds.pint.quantify()
ds_pint

Magnitude,[46.495 46.282 46.005 45.852 45.939 46.38 46.743 46.927 46.968 46.875 46.841 46.908 46.962 47.013 47.247 47.474 47.496 47.357 46.871 46.096 45.925 45.8 45.548 45.135 44.782 44.738 44.641 44.889 44.872 44.781 44.648 44.652 44.479 44.25 43.986 43.823 43.74 43.732 43.789 43.781 44.028 44.439 45.062 45.582 45.899 46.237 46.43 46.568 46.657 47.062 47.532 47.515 47.264 47.435 47.497 47.64 47.522 47.763 48.315 48.453 48.421 48.253 48.433 49.068 49.341 49.445 49.333 49.071 48.61 48.262 48.107 47.885 47.296 47.163 46.938 46.746 47.048 47.032 46.316 46.436 46.638 46.478 45.852 46.215 47.239 47.036 47.295 47.544 47.746 47.612 47.547 47.765 48.016 48.148 48.537 48.721 48.958 49.481 49.835 49.862 49.33 48.847 48.842 49.23 49.569 49.72 49.557 48.871 48.57 48.382 48.03 47.296 46.916 46.733 47.033 47.17 47.298 47.436 47.305 47.322 47.214 47.6 47.23 47.207 47.046 46.947 46.509 45.66 45.255 44.938]
Units,degrees_north
Magnitude,[-27.834 -27.803 -27.608 -27.582 -27.44 -27.492 -27.56 -27.31 -27.371 -27.395 -27.305 -27.208 -27.306 -27.339 -27.245 -27.068 -26.277 -26.053 -25.961 -26.473 -26.984 -27.268 -27.255 -27.747 -27.635 -27.545 -27.389 -26.918 -26.306 -26.136 -25.932 -25.779 -26.04 -26.154 -26.342 -26.491 -26.728 -26.718 -26.642 -26.519 -26.261 -26.272 -26.414 -26.902 -27.126 -27.472 -27.886 -28.15 -27.753 -27.331 -27.132 -27.324 -27.302 -27.292 -27.293 -27.518 -27.752 -27.572 -27.575 -27.729 -27.834 -28.14 -29.078 -29.455 -29.392 -28.376 -27.815 -27.514 -27.465 -27.768 -28.752 -29.627 -29.656 -29.642 -29.515 -29.417 -28.878 -28.183 -28.906 -30.119 -30.458 -31.286 -31.258 -30.578 -30.859 -31.293 -30.954 -30.619 -29.694 -29.126 -28.697 -28.367 -27.909 -27.52 -27.531 -27.685 -27.57 -27.174 -27.232 -27.788 -27.069 -26.75 -27.245 -27.45 -26.982 -26.369 -25.233 -25.562 -25.746 -25.635 -25.364 -25.676 -25.638 -25.225 -24.772 -24.57 -24.612 -24.231 -23.194 -21.768 -19.957 -18.412 -17.054 -15.764 -14.512 -13.757 -14.31 -15.571 -15.605 -14.775]
Units,degrees_east
Magnitude,[[4.400000095367432 9.5 19.399999618530273 ... 1429.0 1478.9000244140625 1505.0] [6.599999904632568 9.399999618530273 19.100000381469727 ... 1428.800048828125 1479.0 1495.9000244140625] [7.800000190734863 9.600000381469727 19.299999237060547 ... 1429.0999755859375 1479.0999755859375 1508.300048828125] ... [11.0 19.399999618530273 29.200000762939453 ... 1479.0999755859375 1507.5999755859375 nan] [11.300000190734863 19.600000381469727 29.600000381469727 ... 1479.0 1504.4000244140625 nan] [10.899999618530273 19.600000381469727 29.299999237060547 ... 1479.0 1508.5 nan]]
Units,decibar
Magnitude,[[0.699999988079071 5.800000190734863 15.699999809265137 ... 1425.300048828125 1475.199951171875 1501.300048828125] [2.700000047683716 5.5 15.199999809265137 ... 1424.9000244140625 1475.0999755859375 1492.0] [3.5999999046325684 5.400000095367432 15.100000381469727 ... 1424.9000244140625 1474.9000244140625 1504.0999755859375] ... [4.5 12.899999618530273 22.700000762939453 ... 1472.5999755859375 1501.0999755859375 nan] [4.800000190734863 13.100000381469727 23.100000381469727 ... 1472.5 1497.9000244140625 nan] [4.5 13.199999809265137 22.899999618530273 ... 1472.5999755859375 1502.0999755859375 nan]]
Units,decibar
Magnitude,[[2.4000000953674316 2.4000000953674316 2.4000000953674316 ... 2.4000000953674316 2.4000000953674316 2.4000000953674316] [2.4000000953674316 2.4000000953674316 2.4000000953674316 ... 2.4000000953674316 2.4000000953674316 2.4000000953674316] [2.4000000953674316 2.4000000953674316 2.4000000953674316 ... 2.4000000953674316 2.4000000953674316 2.4000000953674316] ... [2.4000000953674316 2.4000000953674316 2.4000000953674316 ... 2.4000000953674316 2.4000000953674316 nan] [2.4000000953674316 2.4000000953674316 2.4000000953674316 ... 2.4000000953674316 2.4000000953674316 nan] [2.4000000953674316 2.4000000953674316 2.4000000953674316 ... 2.4000000953674316 2.4000000953674316 nan]]
Units,decibar
Magnitude,[[14.211000442504883 14.213000297546387 14.21399974822998 ... 3.6470000743865

We compute again sigma0, using the `ds_pint` dataset, i.e. variables have a physical dimension

In [11]:
SA = gsw.SA_from_SP(SP=ds_pint.psal_adjusted, p=ds_pint.pres_adjusted, lon=ds_pint.longitude, lat=ds_pint.latitude)
CT = gsw.CT_from_t(SA=SA, t=ds_pint.temp_adjusted, p=ds_pint.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)
sigma0

Magnitude,[[26.647584647258782 26.64656536211237 26.646678569027927 ... 27.77078681910848 27.77467072680065 27.777566090485834] [26.484985859811104 26.484098987420566 26.48432004876031 ... 27.76600562042495 27.769412366010783 27.768950816906] [26.25518806976038 26.254496633565395 26.254887757999086 ... 27.760357802890212 27.764789371196002 27.769248889473374] ... [25.31136543137609 25.316743616248004 25.429988380211398 ... 27.7741477682996 27.76859075800735 nan] [25.30657435108992 25.30409728120253 25.297511083799236 ... 27.78551733197878 27.777782531053845 nan] [25.43166004223758 25.431551606217454 25.432515868475548 ... 27.7946224994771 27.796648720176563 nan]]
Units,kilogram/meter3


gsw-xarray converts the units (if necessary) when using pint quantities:

In [12]:
# start to convert the pressure into Pascal
pressure_in_pascal = ds_pint.pres_adjusted.pint.to('Pa')
pressure_in_pascal

Magnitude,[[7000.0 58000.0 157000.0 ... 14253000.0 14752000.0 15013000.0] [27000.0 55000.0 152000.0 ... 14249000.0 14751000.0 14920000.0] [36000.0 54000.0 151000.0 ... 14249000.0 14749000.0 15041000.0] ... [45000.0 129000.0 227000.0 ... 14726000.0 15011000.0 nan] [48000.0 131000.0 231000.0 ... 14725000.0 14979000.0 nan] [45000.0 132000.0 229000.0 ... 14726000.0 15021000.0 nan]]
Units,pascal


Compute again density, using the pressure in Pascal. No worries as the conversion to dbar is automatic!

In [13]:
SA = gsw.SA_from_SP(SP=ds_pint.psal_adjusted, p=pressure_in_pascal, lon=ds_pint.longitude, lat=ds_pint.latitude)
CT = gsw.CT_from_t(SA=SA, t=ds_pint.temp_adjusted, p=ds_pint.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)
sigma0

Magnitude,[[26.647584647258782 26.646565362110323 26.646678569027927 ... 27.770786819079376 27.77467072680065 27.77756609045673] [26.484985859811104 26.484098987420566 26.48432004876031 ... 27.76600562042495 27.769412366010783 27.768950816906] [26.25518806976038 26.254496633565395 26.254887757999086 ... 27.760357802890212 27.76478937116667 27.769248889473374] ... [25.31136543137609 25.316743616248004 25.429988380202303 ... 27.7741477682996 27.76859075800735 nan] [25.30657435108833 25.304097281203667 25.297511083790823 ... 27.78551733197878 27.777782530944023 nan] [25.43166004223758 25.431551606217454 25.432515868475548 ... 27.7946224994771 27.796648720176563 nan]]
Units,kilogram/meter3


## Using the accessor to simplify the workflow
### Common case

gsw-xarray adds ths `gsw` accessor to datasets. This accessor makes it easy to run the gsw functions on variables from a dataset.

A first solution is to use the accessor exactly as when using gsw:

In [14]:
ds.gsw.SA_from_SP(SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude)

<xarray.DataArray 'SA' (n_prof: 130, n_levels: 56)>
array([[35.81755188, 35.81657471, 35.81659823, ..., 35.08564178,
        35.08162835, 35.0796239 ],
       [35.80757275, 35.8065829 , 35.80559104, ..., 35.09481071,
        35.09079742, 35.08778979],
       [36.03182228, 36.03082693, 36.0308492 , ..., 35.10502264,
        35.1010208 , 35.10101815],
       ...,
       [35.79041046, 35.77693492, 35.76322478, ..., 35.20835127,
        35.19176101,         nan],
       [35.78753964, 35.78651775, 35.77824776, ..., 35.25467841,
        35.22770745,         nan],
       [35.86339942, 35.86237223, 35.85926688, ..., 35.27570263,
        35.26426658,         nan]])
Dimensions without coordinates: n_prof, n_levels
Attributes:
    standard_name:  sea_water_absolute_salinity
    units:          g/kg

This is however not very useful... A better option is to simply give the name of the variables from the dataset:

In [15]:
ds.gsw.SA_from_SP(SP='psal_adjusted', p='pres_adjusted', lon='longitude', lat='latitude')

<xarray.DataArray 'SA' (n_prof: 130, n_levels: 56)>
array([[35.81755188, 35.81657471, 35.81659823, ..., 35.08564178,
        35.08162835, 35.0796239 ],
       [35.80757275, 35.8065829 , 35.80559104, ..., 35.09481071,
        35.09079742, 35.08778979],
       [36.03182228, 36.03082693, 36.0308492 , ..., 35.10502264,
        35.1010208 , 35.10101815],
       ...,
       [35.79041046, 35.77693492, 35.76322478, ..., 35.20835127,
        35.19176101,         nan],
       [35.78753964, 35.78651775, 35.77824776, ..., 35.25467841,
        35.22770745,         nan],
       [35.86339942, 35.86237223, 35.85926688, ..., 35.27570263,
        35.26426658,         nan]])
Dimensions without coordinates: n_prof, n_levels
Attributes:
    standard_name:  sea_water_absolute_salinity
    units:          g/kg

It is even possible to go 1 step further and rely on the usage of standard name! In this case, you don't need to provide any argument for the variables with the proper standard name.

With this method, it is way faster to compute the density:

In [16]:
# WITHOUT any detection
SA = gsw.SA_from_SP(SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude)
CT = gsw.CT_from_t(SA=SA, t=ds.temp_adjusted, p=ds.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)

# WITH autodetection
ds = ds.merge(ds.gsw.SA_from_SP())
ds = ds.merge(ds.gsw.CT_from_t())
ds = ds.merge(ds.gsw.sigma0())

You can also use brackets to get either 1 or multiple variables computed:

In [17]:
ds.gsw['sigma0'] # Returns a DataArray
ds.gsw[['sigma0']] # Returns a Dataset
ds.gsw[['sigma0', 'alpha', 'beta', 'sigma1', 'rho']] # With multiple outputs

<xarray.Dataset>
Dimensions:  (n_prof: 130, n_levels: 56)
Dimensions without coordinates: n_prof, n_levels
Data variables:
    sigma0   (n_prof, n_levels) float64 26.65 26.65 26.65 ... 27.79 27.8 nan
    alpha    (n_prof, n_levels) float64 0.0002081 0.0002082 ... 0.0001458 nan
    beta     (n_prof, n_levels) float64 0.0007438 0.0007437 ... 0.0007505 nan
    sigma1   (n_prof, n_levels) float64 31.01 31.01 31.01 ... 32.37 32.38 nan
    rho      (n_prof, n_levels) float64 1.027e+03 1.027e+03 ... 1.035e+03 nan

Of course any kind of mixture between all the solutions is possible:

In [18]:
# Give a value for SP
# Take p from dataset
# Automatically get lon and lat based on standard names
ds.gsw.SA_from_SP(SP=35, p='pres_adjusted')

<xarray.DataArray 'SA' (n_prof: 130, n_levels: 56)>
array([[35.16540604, 35.16543255, 35.16545565, ..., 35.16805541,
        35.16807052, 35.16807842],
       [35.16541107, 35.16542513, 35.16544852, ..., 35.16805194,
        35.16806717, 35.1680723 ],
       [35.1654079 , 35.16541651, 35.16543824, ..., 35.16804504,
        35.16806022, 35.16806909],
       ...,
       [35.16533263, 35.16534798, 35.16535502, ..., 35.16820029,
        35.16823002,         nan],
       [35.16532498, 35.16533771, 35.16534623, ..., 35.16817466,
        35.16820349,         nan],
       [35.1653088 , 35.16532382, 35.16533425, ..., 35.16819766,
        35.16823758,         nan]])
Dimensions without coordinates: n_prof, n_levels
Attributes:
    standard_name:  sea_water_absolute_salinity
    units:          g/kg

And it is also possible to use automatic discovery of argument with pint datasets:

In [19]:
ds_pint.gsw.SA_from_SP()

Magnitude,[[35.81755188097689 35.81657470990524 35.81659823422674 ... 35.08564177863203 35.08162835277999 35.07962390243301] [35.80757275281334 35.806582899327445 35.80559104019445 ... 35.09481071496229 35.0907974217668 35.087789789910715] [36.03182228097798 36.030826926123865 36.03084919985498 ... 35.105022642266206 35.101020798151254 35.10101814856906] ... [35.79041046332032 35.776934917452046 35.76322477733858 ... 35.2083512721116 35.19176100766081 nan] [35.78753963934035 35.78651775263226 35.778247759076685 ... 35.25467841261591 35.22770745257018 nan] [35.86339941748716 35.862372233707966 35.859266875293244 ... 35.27570263128744 35.264266577481074 nan]]
Units,gram/kilogram


### Case with some argument without standard name

Some functions have argument without any standard name. In this case, it is possible to refer to these arguments using gsw-xarray options.

Let's take the function `gsw.SP_salinometer` that has 2 arguments: `Rt` without standard name, and `t` the in situ temperature.

A 1st option is to explicitely provide a value or the name from the dataset (we will create some fake data for the purpose of this example):

In [20]:
ds['salinometer_Rt'] = 35

In [21]:
ds.gsw.SP_salinometer(Rt='salinometer_Rt')

<xarray.DataArray 'SP' (n_prof: 130, n_levels: 56)>
array([[14852.44767448, 14852.37691945, 14852.34156055, ...,
        15304.61363721, 15308.13947294, 15310.42597351],
       [14827.44354407, 14827.40896543, 14827.51267175, ...,
        15298.31168787, 15301.51050946, 15302.44067949],
       [14768.33013327, 14768.33013327, 14768.33013327, ...,
        15291.27112184, 15294.91511554, 15297.07554699],
       ...,
       [14673.397394  , 14675.12423965, 14689.35099574, ...,
        15258.27973762, 15261.63122146,            nan],
       [14673.09995293, 14672.86208721, 14672.77288815, ...,
        15246.89048002, 15252.84600545,            nan],
       [14680.80248747, 14680.83244267, 14681.16223826, ...,
        15243.57738327, 15248.32719727,            nan]])
Dimensions without coordinates: n_prof, n_levels
Attributes:
    standard_name:    sea_water_practical_salinity
    units:            1
    reference_scale:  PSS-78

A 2nd solution is to use gsw-xarray option with function `set_non_cf_name`. This way if you need to compute multiple times functions that use these arguments without a standard name, you only need to provide once the mapping.

In [22]:
gsw.set_non_cf_name?

Init signature: gsw.set_non_cf_name(**kwargs)
Docstring:     
Set `non_cf_name` options for gsw_xarray in a controlled context.

Parameters
----------
Provide the name in dataset of arguments that don't have a standard name
e.g. entropy='entropy_name_in_ds', SA_seaice='salinity_of_sea_ice_name_in_ds'

Using `set_non_cf_name` is equivalent to using `set_options`
with argument 'set_non_cf_name', but is provided as a shorter method.

You can use `set_non_cf_name` either as a context manager (using `with`)
or to set global options
File:           ~/Documents/Education/PhD/Dev-prog/gsw-xarray/gsw_xarray/_options.py
Type:           type
Subclasses:     

Using a context manager:

In [23]:
with gsw.set_non_cf_name(Rt='salinometer_Rt'):
    ds.gsw.SP_salinometer()

Or globally:

In [24]:
gsw.set_non_cf_name(Rt='salinometer_Rt')
ds.gsw.SP_salinometer()

<xarray.DataArray 'SP' (n_prof: 130, n_levels: 56)>
array([[14852.44767448, 14852.37691945, 14852.34156055, ...,
        15304.61363721, 15308.13947294, 15310.42597351],
       [14827.44354407, 14827.40896543, 14827.51267175, ...,
        15298.31168787, 15301.51050946, 15302.44067949],
       [14768.33013327, 14768.33013327, 14768.33013327, ...,
        15291.27112184, 15294.91511554, 15297.07554699],
       ...,
       [14673.397394  , 14675.12423965, 14689.35099574, ...,
        15258.27973762, 15261.63122146,            nan],
       [14673.09995293, 14672.86208721, 14672.77288815, ...,
        15246.89048002, 15252.84600545,            nan],
       [14680.80248747, 14680.83244267, 14681.16223826, ...,
        15243.57738327, 15248.32719727,            nan]])
Dimensions without coordinates: n_prof, n_levels
Attributes:
    standard_name:    sea_water_practical_salinity
    units:            1
    reference_scale:  PSS-78

## Note on performance

It is still very slightly faster to provide the arguments than rely on autodetect, however for large data sets this difference should be negligible compared to the internal computation time of the gsw functions.

In [38]:
# We chunk ds with dask to focus on overhead time
ds = ds.chunk()

In [39]:
%%timeit
gsw.SA_from_SP(SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude)

1.26 ms ± 58.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [40]:
%%timeit
ds.gsw.SA_from_SP(SP='psal_adjusted', p='pres_adjusted', lon='longitude', lat='latitude')

1.53 ms ± 73.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [41]:
%%timeit
ds.gsw.SA_from_SP()

1.49 ms ± 45.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
